In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from matplotlib import cm
import operator
import matplotlib.ticker as ticker
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.lines as mlines

sns.set_context('paper')
sns.set_style('white')
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
path = r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque'
savepath = os.path.join(path, 'figures', 'after_QC')

In [ ]:
dat = pd.read_csv(os.path.join(path, 'unionizes', 'plaque_unionizes.csv'))
dat = dat[dat['control'] == False]

In [ ]:
dat.head()

In [ ]:
len(dat['image_series_id'].unique())

In [ ]:
dat['age_group'].unique()

In [ ]:
dat['count_per_mm_3'] = dat['sum_plaques']/dat['volume'] #to convert to plaques per mm^3
dat['avg_plaque_volume'] = dat['avg_plaque_volume']/1000 #to convert to cubic microns

In [ ]:
dataset = pd.read_csv(os.path.join(path, '_final_plaque_dataset.csv'))
print(len(dataset))

In [ ]:
dat['mouse_line'].unique()

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
summary_structures = st.get_structures_by_set_id([167587189])
summary_structure_ids = [item['id'] for item in summary_structures]
ss_acronyms = [item['acronym'] for item in summary_structures]
ont_order = [item['graph_order'] for item in summary_structures]
ont_order_dict = dict(zip(ss_acronyms, ont_order))
print(len(summary_structure_ids))


In [ ]:
coarse_structures = st.get_structures_by_set_id([687527670])
coarse_structure_ids = [item['id'] for item in coarse_structures]
coarse_structure_acronyms = [item['acronym'] for item in coarse_structures]
ft = st.get_structures_by_acronym(['fiber tracts'])
coarse_structure_ids.append(ft[0]['id'])
coarse_structure_acronyms.append(ft[0]['acronym'])
ont_order = [item['graph_order'] for item in coarse_structures]
ont_order.append(ft[0]['graph_order'])
oo_dict_coarse = dict(zip(coarse_structure_acronyms, ont_order))
print(len(coarse_structure_ids))

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()
ia_map = structure_tree.get_id_acronym_map()
ai_map = {value:key for key, value in ia_map.iteritems()}

In [ ]:
structures = summary_structure_ids + coarse_structure_ids + [997]

In [ ]:
dataset['image_series_id'] = [int(value) for value in dataset['Link to image series']]

In [ ]:
[isid for isid in dat['image_series_id'] if isid not in dataset['image_series_id'].values]

In [ ]:
dataset.groupby(['mouse_line', 'age_group', 'Sex'])['image_series_id'].count()

In [ ]:
dat['structure_id'] = [ia_map[structure_acronym] for structure_acronym in dat['structure_acronym']]

In [ ]:
def get_structure_data(age, structure, q_type):
    data = dat
    if q_type == 'density':
        data = data.rename(columns = {'plaque_density': 'plaque'})
    elif q_type == 'count':
        data = data.rename(columns = {'count_per_mm_3': 'plaque'})
    elif q_type == 'volume':
        data = data.rename(columns = {'avg_plaque_volume': 'plaque'})
    structure_id = ia_map[structure]
    groups = []
    plaques = []
    isids_carrythrough = []
    for group in dataset[dataset['age_group'] == age]['mouse_line'].unique():
        isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
        for isid in isids:
            groups.append(group)
            if len(data[(data['structure_id'] == structure_id) & (
                data['image_series_id'] == isid)]['plaque']) > 0:
                plaques.append(data[(data['structure_id'] == structure_id) & 
                                         (data['image_series_id'] == isid)]
                                     ['plaque'].values[0])
            else:
                print(isid)
                print(ai_map[structure_id])
                plaques.append(0)
                    
            isids_carrythrough.append(isid)
    structuredat = pd.DataFrame({'age': age, 'structure': structure, 
                                 'group': groups, 'plaques': plaques,
                                'image_series_id': isids_carrythrough})
    return structuredat

In [ ]:
print(len(dataset['image_series_id'].unique()))

In [ ]:
mouse_line_colors = ['#8da0cb', '#66c2a5', '#fc8d62']
mouse_lines = ['APP/PS1', 'Tg2576', 'hAPP-J20']

In [ ]:
structure = 'root'
structure_id = ia_map[structure]
order = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']

ages = []
groups = []
plaque_density = []
isids_carrythrough = []
for age in dataset['age_group'].unique():
    for group in dataset[dataset['age_group'] == age]['mouse_line'].unique():
        isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
        for isid in isids:
            ages.append(age)
            groups.append(group)
            plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                             (dat['image_series_id'] == isid)]
                                         ['plaque_density'].values[0])
            isids_carrythrough.append(isid)
density_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque density': plaque_density,
                             'image_series_id': isids_carrythrough})
density_dat['plaque density'] = [density*100 for density in density_dat['plaque density']]

ages = []
groups = []
plaque_count = []
isids_carrythrough = []
for age in dataset['age_group'].unique():
    for group in dataset[dataset['age_group'] == age]['mouse_line'].unique():
        isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
        for isid in isids:
            ages.append(age)
            groups.append(group)
            plaque_count.append(dat[(dat['structure_id'] == structure_id) & 
                                             (dat['image_series_id'] == isid)]
                                         ['count_per_mm_3'].values[0])
            isids_carrythrough.append(isid)
count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque count': plaque_count,
                             'image_series_id': isids_carrythrough})


fig, ax = plt.subplots(1, 2, figsize = (8, 4))
g = sns.swarmplot('age', 'plaque density', hue = 'group', data=density_dat, ax=ax[0],
                 order = order, palette = mouse_line_colors, hue_order = mouse_lines)
#g.set_ylim([0, 0.016])
g.legend_.remove()
g.set_ylabel('Plaque Percent Volume')
g.set_title('Brain-Wide Plaque Density')
h = sns.swarmplot('age', 'plaque count', hue = 'group', 
                  data=count_dat, ax=ax[1], order = order,
                 palette = mouse_line_colors, hue_order = mouse_lines)
h.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
g.legend(loc=2, frameon = False)
g.set_xlabel("")
h.legend(loc=2, frameon = False)
h.set_title('Brain-Wide Plaque Count')
h.set_xlabel("")
#fig.subplots_adjust(wspace = 0.3)
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count brain-wide 3 lines root.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
sns.distplot(density_dat[density_dat['group'] == 'APP/PS1']['plaque density'])

In [ ]:
structure = 'root'
structure_id = ia_map[structure]
order = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']

sexes = []
ages = []
groups = []
plaque_density = []
isids_carrythrough = []
for age in dataset['age_group'].unique():
    for group in dataset[dataset['age_group'] == age]['mouse_line'].unique():
        isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
        for isid in isids:
            sexes.append( dataset[(dataset['mouse_line'] == group) & 
                                  (dataset['age_group'] == age) &
                                 (dataset['image_series_id'] == isid)]['Sex'].values)
            ages.append(age)
            groups.append(group)
            plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                         (dat['image_series_id'] == isid)]
                                     ['plaque_density'].values[0])
            isids_carrythrough.append(isid)
density_dat = pd.DataFrame({'age': ages, 'sex': sexes, 'structure': structure, 
                             'group': groups, 'plaque density': plaque_density,
                             'image_series_id': isids_carrythrough})
density_dat['plaque density'] = [density*100 for density in density_dat['plaque density']]

sexes = []
ages = []
groups = []
plaque_count = []
isids_carrythrough = []
for age in dataset['age_group'].unique():
    for group in dataset[dataset['age_group'] == age]['mouse_line'].unique():
        isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
        for isid in isids:
            sexes.append( dataset[(dataset['mouse_line'] == group) & 
                                  (dataset['age_group'] == age) &
                                 (dataset['image_series_id'] == isid)]['Sex'].values)
            ages.append(age)
            groups.append(group)
            plaque_count.append(dat[(dat['structure_id'] == structure_id) & 
                                         (dat['image_series_id'] == isid)]
                                     ['count_per_mm_3'].values[0])
            isids_carrythrough.append(isid)
count_dat = pd.DataFrame({'age': ages, 'sex': sexes, 'structure': structure, 
                             'group': groups, 'plaque count': plaque_count,
                             'image_series_id': isids_carrythrough})

fig, ax = plt.subplots(1, 2, figsize = (8, 4))
h = sns.swarmplot('age', 'plaque density', hue = 'group', data=density_dat[density_dat['sex'] == 'M'], 
                  ax=ax[0], order = order, palette = mouse_line_colors, alpha = 0.7,
                  hue_order = mouse_lines)
h.legend_.remove()
g = sns.swarmplot('age', 'plaque density', hue = 'group', data=density_dat[density_dat['sex'] == 'F'], 
                  ax=ax[0], order = order, palette = mouse_line_colors, marker=ur'$\u2640$', s=7,
                  hue_order = mouse_lines)
#g.set_ylim([0, 0.016])
g.legend_.remove()
g.set_ylabel('Percent Plaque Coverage')
g.set_title('Brain-Wide Plaque Density')
i = sns.swarmplot('age', 'plaque count', hue = 'group', data=count_dat[count_dat['sex'] == 'M'], 
                  ax=ax[1], order = order, palette = mouse_line_colors, alpha = 0.7,
                  hue_order = mouse_lines)
i.legend_.remove()
j = sns.swarmplot('age', 'plaque count', hue = 'group', data=count_dat[count_dat['sex'] == 'F'], 
                  ax=ax[1], order = order, palette = mouse_line_colors, marker=ur'$\u2640$', s=7,
                  hue_order = mouse_lines)
i.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')

handles, labels = ax[0].get_legend_handles_labels()
g.legend(handles[0:3], labels[0:3],loc=2, borderaxespad=0., frameon = False)
g.set_xlabel("")
i.legend(handles[0:3], labels[0:3],loc=2, borderaxespad=0., frameon = False)
i.set_title('Brain-Wide Plaque Count')
i.set_xlabel("")
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count brain-wide 3 lines MF.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
sexes = []
ages = []
groups = []
mean_volume = []
isids_carrythrough = []
for age in dataset['age_group'].unique():
    for group in dataset[dataset['age_group'] == age]['mouse_line'].unique():
        isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
        for isid in isids:
            sexes.append( dataset[(dataset['mouse_line'] == group) & 
                                  (dataset['age_group'] == age) &
                                 (dataset['image_series_id'] == isid)]['Sex'].values)
            ages.append(age)
            groups.append(group)
            mean_volume.append(dat[(dat['structure_id'] == structure_id) & 
                                         (dat['image_series_id'] == isid)]
                                     ['avg_plaque_volume'].values[0])
            isids_carrythrough.append(isid)
volume_dat = pd.DataFrame({'age': ages, 'sex': sexes, 'structure': structure, 
                             'group': groups, 'mean plaque volume': mean_volume,
                             'image_series_id': isids_carrythrough})

fig, ax = plt.subplots()
k = sns.swarmplot('age', 'mean plaque volume', hue = 'group', data=volume_dat[volume_dat['sex'] == 'M'], 
                  ax=ax, order = order, palette = mouse_line_colors, alpha = 0.7,
                  hue_order = mouse_lines)
k.legend_.remove()
j = sns.swarmplot('age', 'mean plaque volume', hue = 'group', data=volume_dat[volume_dat['sex'] == 'F'], 
                  ax=ax, order = order, palette = mouse_line_colors, marker=ur'$\u2640$', s=7,
                  hue_order = mouse_lines)
j.set_ylabel('Mean Plaque Volume (mm$\mathregular{^{3}}$)')
j.legend_.remove()
k.legend(handles[0:3], labels[0:3],loc=2, borderaxespad=0.)
k.set_title('Plaque Volume')
k.set_xlabel("")

In [ ]:
def find_density_by_structure(structure):
    structure_id = ia_map[structure]
    sexes = []
    ages = []
    groups = []
    plaque_density = []
    isids_carrythrough = []
    for age in dataset['age_group'].unique():
        for group in dataset[dataset['age_group'] == age]['mouse_line'].unique():
            isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
            for isid in isids:
                sexes.append( dataset[(dataset['mouse_line'] == group) & 
                                      (dataset['age_group'] == age) &
                                     (dataset['image_series_id'] == isid)]['Sex'].values)
                ages.append(age)
                groups.append(group)
                plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                             (dat['image_series_id'] == isid)]
                                         ['plaque_density'].values[0])
                isids_carrythrough.append(isid)
    density_dat = pd.DataFrame({'age': ages, 'sex': sexes, 'structure': structure, 
                                 'group': groups, 'plaque density': plaque_density,
                                 'image_series_id': isids_carrythrough})
    density_dat['plaque density'] = [density*100 for density in density_dat['plaque density']]
    return(density_dat)

def find_count_by_structure(structure):
    structure_id = ia_map[structure]
    sexes = []
    ages = []
    groups = []
    plaque_count = []
    isids_carrythrough = []
    for age in dataset['age_group'].unique():
        for group in dataset[dataset['age_group'] == age]['mouse_line'].unique():
            isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
            for isid in isids:
                sexes.append( dataset[(dataset['mouse_line'] == group) & 
                                      (dataset['age_group'] == age) &
                                     (dataset['image_series_id'] == isid)]['Sex'].values)
                ages.append(age)
                groups.append(group)
                plaque_count.append(dat[(dat['structure_id'] == structure_id) & 
                                             (dat['image_series_id'] == isid)]
                                         ['count_per_mm_3'].values[0])
                isids_carrythrough.append(isid)
    count_dat = pd.DataFrame({'age': ages, 'sex': sexes, 'structure': structure, 
                                 'group': groups, 'plaque count': plaque_count,
                                 'image_series_id': isids_carrythrough})
    return(count_dat)

In [ ]:
order = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
root_d = find_density_by_structure('root')
root_c = find_count_by_structure('root')

iso_d = find_density_by_structure('Isocortex')
iso_c = find_count_by_structure('Isocortex')

hpc_d = find_density_by_structure('HPF')
hpc_c = find_count_by_structure('HPF')

fig, ax = plt.subplots(4, 1, figsize = (3, 10))
a = sns.swarmplot('age', 'plaque density', hue = 'group', data=root_d[root_d['sex'] == 'M'], 
                  ax=ax[1], order = order, palette = mouse_line_colors, alpha = 0.7,
                  hue_order = mouse_lines)
a.legend_.remove()
a = sns.swarmplot('age', 'plaque density', hue = 'group', data=root_d[root_d['sex'] == 'F'], 
                  ax=ax[1], order = order, palette = mouse_line_colors, marker=ur'$\u2640$', s=7,
                  hue_order = mouse_lines)
ax[1].axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
a.set_ylabel('Percent Plaque Coverage')
a.set_xlabel("")
handles, labels = ax[0].get_legend_handles_labels()
a.legend(handles[0:3], labels[0:3],loc=2, borderaxespad=0., frameon = False)
a.set_title('Brain-Wide Plaque Density')

b = sns.swarmplot('age', 'plaque count', hue = 'group', data=root_c[root_c['sex'] == 'M'], 
                  ax=ax[0], order = order, palette = mouse_line_colors, alpha = 0.7,
                  hue_order = mouse_lines)
b.legend_.remove()
b = sns.swarmplot('age', 'plaque count', hue = 'group', data=root_c[root_c['sex'] == 'F'], 
                  ax=ax[0], order = order, palette = mouse_line_colors, marker=ur'$\u2640$', s=7,
                  hue_order = mouse_lines)
ax[0].axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
b.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
b.set_xlabel("")
b.set_title('Brain-Wide Plaque Count')
b.legend_.remove()

c = sns.swarmplot('age', 'plaque density', hue = 'group', data=iso_d[iso_d['sex'] == 'M'], 
                  ax=ax[2], order = order, palette = mouse_line_colors, alpha = 0.7,
                  hue_order = mouse_lines)
c.legend_.remove()
c = sns.swarmplot('age', 'plaque density', hue = 'group', data=iso_d[iso_d['sex'] == 'F'], 
                  ax=ax[2], order = order, palette = mouse_line_colors, marker=ur'$\u2640$', s=7,
                  hue_order = mouse_lines)
ax[2].axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
c.set_ylabel('Percent Plaque Coverage')
c.set_xlabel("")
c.legend_.remove()
c.set_title('Plaque Density in Isocortex')
e = sns.swarmplot('age', 'plaque density', hue = 'group', data=hpc_d[hpc_d['sex'] == 'M'], 
                  ax=ax[3], order = order, palette = mouse_line_colors, alpha = 0.7,
                  hue_order = mouse_lines)
e.legend_.remove()
e = sns.swarmplot('age', 'plaque density', hue = 'group', data=hpc_d[hpc_d['sex'] == 'F'], 
                  ax=ax[3], order = order, palette = mouse_line_colors, marker=ur'$\u2640$', s=7,
                  hue_order = mouse_lines)
ax[3].axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
e.set_ylabel('Percent Plaque Coverage')
e.set_xlabel("")
e.legend_.remove()
e.set_title('Plaque Density in Hippocampus')

plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count brain-wide iso hipp 3 lines MF.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
root_d.head()

In [ ]:
meddat_root.head()

In [ ]:
order = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
root_d = find_density_by_structure('root')
meddat_root = root_d.groupby(['group', 'age']).median().reset_index()
root_c = find_count_by_structure('root')
meddat_root_c = root_c.groupby(['group', 'age']).median().reset_index()

iso_d = find_density_by_structure('Isocortex')
meddat_iso = iso_d.groupby(['group', 'age']).median().reset_index()
hpc_d = find_density_by_structure('HPF')
meddat_hpc = hpc_d.groupby(['group', 'age']).median().reset_index()

fig, ax = plt.subplots(2, 1, figsize = (2.5, 4), sharex = True)

a = sns.stripplot('age', 'plaque density', hue = 'group', data=root_d, 
                  ax=ax[1], order = order, palette = mouse_line_colors, alpha = 0.5,
                  hue_order = mouse_lines)
a = sns.pointplot('age', 'plaque density', hue = 'group', data=meddat_root, marker = None,
                  ax=ax[1], order = order, palette = mouse_line_colors, 
                  hue_order = mouse_lines)
a.legend_.remove()
#ax[1].axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
ax[1].set_yscale('log')
ax[1].set_ylim([10e-4, 10])
a.set_ylabel('Plaque Load (%) \n (log scale)')
a.set_xlabel("")
a.set_title('Plaque Density')

b = sns.stripplot('age', 'plaque count', hue = 'group', data=root_c, 
                  ax=ax[0], order = order, palette = mouse_line_colors, alpha = 0.5,
                  hue_order = mouse_lines)
b = sns.pointplot('age', 'plaque count', hue = 'group', data=meddat_root_c, marker = None,
                  ax=ax[0], order = order, palette = mouse_line_colors, 
                  hue_order = mouse_lines)
b.legend_.remove()
#ax[0].axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
b.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
b.set_xlabel("")
b.set_title('Plaque Count')

handles, labels = ax[0].get_legend_handles_labels()

plt.xticks(rotation = 45)
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count brain-wide 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
fig, ax = plt.subplots()
ax.legend(handles[0:3], labels[0:3],loc='upper center', borderaxespad=0., markerscale = 0.6, ncol = 3,
        bbox_to_anchor = (0.5, -0.4))
plt.savefig(os.path.join(savepath, 'mouse_lines_legend_horizontal.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
order = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
root_d = find_density_by_structure('root')
meddat_root = root_d.groupby(['group', 'age']).median().reset_index()
root_c = find_count_by_structure('root')
meddat_root_c = root_c.groupby(['group', 'age']).median().reset_index()

iso_d = find_density_by_structure('Isocortex')
meddat_iso = iso_d.groupby(['group', 'age']).median().reset_index()
hpc_d = find_density_by_structure('HPF')
meddat_hpc = hpc_d.groupby(['group', 'age']).median().reset_index()

fig, ax = plt.subplots(2, 1, figsize = (2.5, 4), sharex = True)
c = sns.stripplot('age', 'plaque density', hue = 'group', data=iso_d, 
                  ax=ax[0], order = order, palette = mouse_line_colors, alpha = 0.5,
                  hue_order = mouse_lines)
c = sns.pointplot('age', 'plaque density', hue = 'group', data=meddat_iso, marker = None,
                  ax=ax[0], order = order, palette = mouse_line_colors, 
                  hue_order = mouse_lines)
c.legend_.remove()
#ax[2].axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
ax[0].set_yscale('log')
ax[0].set_ylim([10e-4, 10])
c.set_ylabel('Plaque Load (%) \n (log scale)')
c.set_xlabel("")
c.set_title('Isocortex')
e = sns.stripplot('age', 'plaque density', hue = 'group', data=hpc_d, 
                  ax=ax[1], order = order, palette = mouse_line_colors, alpha = 0.5,
                  hue_order = mouse_lines)
e = sns.pointplot('age', 'plaque density', hue = 'group', data=meddat_hpc, marker = None,
                  ax=ax[1], order = order, palette = mouse_line_colors, 
                  hue_order = mouse_lines)
e.legend_.remove()

#ax[3].axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
ax[1].set_yscale('log')
ax[1].set_ylim([10e-4, 10])
e.set_ylabel('Plaque Load (%) \n (log scale)')
e.set_xlabel("")
e.set_title('Hippocampus')

plt.xticks(rotation = 45)
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count iso hipp 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
density_dat['group'].unique()

In [ ]:
def get_mouse_line_legend():
    blue = mlines.Line2D([], [], color=mouse_line_colors[0],
                          label='APP/PS1')
    green = mlines.Line2D([], [], color=mouse_line_colors[1],
                          label='Tg2576')
    orange = mlines.Line2D([], [], color=mouse_line_colors[2],
                          label='hAPP-J20')
    return [blue, green, orange]

In [ ]:
density_dat = find_density_by_structure('grey')
g = sns.catplot('age', 'plaque density', hue = 'group', 
                   data=density_dat, kind = 'strip',
                   height = 2, aspect = 2.5, order = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo'],
                  palette = mouse_line_colors, legend = False, alpha = 0.6, label = density_dat['group'].unique(),
                  hue_order = mouse_lines)
h = sns.pointplot('age', 'plaque density', hue = 'group', 
                   data=density_dat, estimator = np.median, markers = 'd',
                   height = 2, aspect = 4, order = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo'],
                  ci = False, palette = mouse_line_colors, legend = False, 
                  hue_order = mouse_lines, yerr = 'err')
h.set(yscale="log")
#ax.set_ylim([0.001, 10])
#g.set_yticklabels(['0.001', '0.01', '0.1', '1', '10'])
plt.title('Whole Brain Plaque Density', y=1.1)
plt.legend(handles=get_mouse_line_legend(), loc=2, bbox_to_anchor = (0, 1.3))
h.set_ylabel('Plaque Percent Volume \n(log10)')
h.set_xlabel('')
plt.savefig(os.path.join(savepath, 'whole brain plaque density by age 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
ccf_colors = ['#70ff71', '#9ad2bd', '#7ed04b', '#8ada87', '#98d6f9', '#8599cc', '#ff7080', '#e64438', '#ff64ff', 
              '#ff9b88','#ff9bcd', '#f0f080', '#cccccc']

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('13 mo', structure_acronym, 'density')], sort = True)
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(1, 3, figsize = (4, 4), sharex = False, sharey = True)
groups = ['APP/PS1', 'Tg2576', 'hAPP-J20'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('plaque density', 'structure', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white', fliersize = 1)
    sns.stripplot('plaque density', 'structure', data = mdf[mdf['group'] == group], alpha = 0.3,
                  palette = ccf_colors, ax=ax[ix], dodge = True)
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        g.set_xlim([0, 0.3])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(0.3))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'APP/PS1':
        g.set_xlim([0, 3])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
        ax[ix].axvline(x=0.3, color = 'grey', linestyle = 'dashed', zorder = -1)
    if group == 'hAPP-J20':
        ax[ix].set_xlabel('Plaque Percent Volume')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.3)
plt.savefig(os.path.join(savepath, 'box-swarm plots major divisions 12 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('13 mo', structure_acronym, 'density')], sort = True)
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(3, 1, figsize = (4, 3.5), sharex = True)
groups = ['APP/PS1', 'Tg2576', 'hAPP-J20'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('structure', 'plaque density', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white', fliersize = 1)
    sns.stripplot('structure', 'plaque density', data = mdf[mdf['group'] == group], alpha = 0.3,
                  palette = ccf_colors, ax=ax[ix], dodge = True)
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        g.set_ylim([0, 0.25])
        ax[ix].yaxis.set_major_locator(ticker.MultipleLocator(0.1))
        ax[ix].yaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_xlabel('')
        ax[ix].set_ylabel('')
    if group == 'APP/PS1':
        g.set_ylim([0, 3])
        ax[ix].yaxis.set_major_locator(ticker.MultipleLocator(1))
        ax[ix].yaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_xlabel('')
        ax[ix].set_ylabel('')
        ax[ix].axhline(y=0.25, color = 'grey', linestyle = 'dashed', zorder = -1)
    if group == 'Tg2576':
        ax[ix].set_ylabel('Plaque Load (%)')
    plt.tight_layout()
    fig.subplots_adjust(hspace = 0.4)
plt.xticks(rotation = 90)
plt.savefig(os.path.join(savepath, 'box-swarm plots major divisions horizontal 12 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('19 mo', structure_acronym, 'density')], sort = True)
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(3, 1, figsize = (4, 3.5), sharex = True)
groups = ['APP/PS1', 'Tg2576', 'hAPP-J20'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('structure', 'plaque density', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white', fliersize = 1)
    sns.stripplot('structure', 'plaque density', data = mdf[mdf['group'] == group], alpha = 0.3,
                  palette = ccf_colors, ax=ax[ix], dodge = True)
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        g.set_ylim([0, 0.7])
        ax[ix].yaxis.set_major_locator(ticker.MultipleLocator(0.2))
        ax[ix].yaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_xlabel('')
        ax[ix].set_ylabel('')
    if group == 'APP/PS1':
        g.set_ylim([0, 3])
        ax[ix].yaxis.set_major_locator(ticker.MultipleLocator(1))
        ax[ix].yaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_xlabel('')
        ax[ix].set_ylabel('')
        ax[ix].axhline(y=0.7, color = 'grey', linestyle = 'dashed', zorder = -1)
    if group == 'Tg2576':
        ax[ix].set_ylabel('Plaque Load (%)')
    plt.tight_layout()
    fig.subplots_adjust(hspace = 0.4)
plt.xticks(rotation = 90)
plt.savefig(os.path.join(savepath, 'box-swarm plots major divisions horizontal 18 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
g.legend?

In [ ]:
order = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
thal_d = find_density_by_structure('TH')
thal_d['sex'] = [sex[0][0] for sex in thal_d['sex'].values]
colors = ['#fdd49e', '#fdbb84', '#fc8d59', '#e34a33', '#b30000']

fig, ax = plt.subplots(figsize = (1, 2))
g = sns.swarmplot('sex', 'plaque density', hue = 'age', data=thal_d[thal_d['group'] == 'hAPP-J20'], 
                  ax=ax, palette = colors, hue_order = order)
ax.axhline(y=0, color = 'grey', linestyle = 'dashed', zorder = -1)
g.set_ylabel('Percent Plaque Coverage')
g.set_xlabel("")
ax.set_ylim([0, 0.05])
g.set_title('Plaque Density in Thalamus \n hAPP-J20 mice', y=1.05)
g.legend(frameon = False, markerscale = 0.5, bbox_to_anchor = (0.9, 1))
plt.savefig(os.path.join(savepath, 'J20_thalamus_by_sex.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

## Plotting Functions

In [ ]:
light_colors = ['#afb8cc', '#b5cec6', '#f9c9b6']

In [ ]:
def make_boxplots(age, structures, title, width, height, ax_split = 0):
    pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
    for structure in structures:
        structure_acronym = ai_map[structure]
        pltdat = pd.concat([pltdat, get_structure_data(age, structure_acronym, 'density')], sort = True)
    mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
    mdf['Percent Plaque Coverage'] = [density*100 for density in mdf['plaque density']]
    
    fig, ax = plt.subplots(3, figsize = (width, height))
    iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
    iso = structure_tree.descendant_ids([iso['id']])[0]
    iso = [structure for structure in iso if structure in summary_structure_ids]
    if structures == iso:
        g = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf[mdf['group'] == 'APP/PS1'], 
                        order = plot_order, dodge = False, ax = ax[0], fliersize = 1)
        for artist in ax[0].artists:
            artist.set_facecolor('None')
        g = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'group', 
                           data = mdf[mdf['group'] == 'APP/PS1'], alpha = 0.3,
                           order = plot_order, palette = mouse_line_colors,
                          hue_order = mouse_lines, ax = ax[0])
        h = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf[mdf['group'] == 'Tg2576'], 
                        order = plot_order, dodge = False, ax = ax[1], fliersize = 1)
        for artist in ax[1].artists:
            artist.set_facecolor('None')
        h = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'group', 
                           data = mdf[mdf['group'] == 'Tg2576'], alpha = 0.3,
                           order = plot_order, palette = mouse_line_colors,
                          hue_order = mouse_lines, ax = ax[1])
        h = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf[mdf['group'] == 'hAPP-J20'], 
                        order = plot_order, dodge = False, ax = ax[2], fliersize = 1)
        for artist in ax[2].artists:
            artist.set_facecolor('None')
        h = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'group', 
                           data = mdf[mdf['group'] == 'hAPP-J20'], alpha = 0.3,
                           order = plot_order, palette = mouse_line_colors,
                          hue_order = mouse_lines, ax = ax[2])
    else:
        g = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf[mdf['group'] == 'APP/PS1'], 
                        dodge = False, ax = ax[0], fliersize = 1)
        for artist in ax[0].artists:
            artist.set_facecolor('None')
        g = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'group', 
                           data = mdf[mdf['group'] == 'APP/PS1'], alpha = 0.3,
                           palette = mouse_line_colors,
                          hue_order = mouse_lines, ax = ax[0])
        h = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf[mdf['group'] == 'Tg2576'], 
                        dodge = False, ax = ax[1], fliersize = 1)
        for artist in ax[1].artists:
            artist.set_facecolor('None')
        h = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'group', 
                           data = mdf[mdf['group'] == 'Tg2576'], alpha = 0.3,
                           palette = mouse_line_colors,
                          hue_order = mouse_lines, ax = ax[1])
        h = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf[mdf['group'] == 'hAPP-J20'], 
                        dodge = False, ax = ax[2], fliersize = 1)
        for artist in ax[2].artists:
            artist.set_facecolor('None')
        h = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'group', 
                           data = mdf[mdf['group'] == 'hAPP-J20'], alpha = 0.3,
                           palette = mouse_line_colors,
                          hue_order = mouse_lines, ax = ax[2])
    plt.xticks(rotation = 90)
    plt.subplots_adjust(top=2)
    if ax_split == 0:
        yval = np.max(mdf[mdf['group'].isin(['hAPP-J20', 'Tg2576'])]['Percent Plaque Coverage'])
    else:
        yval = ax_split
    ticks = yval/5
    ax[0].set_xlabel("")
    ax[0].set_xticklabels("")
    ax[0].legend_.remove()
    ax[0].set_ylabel("")
    ax[0].axhline(y=yval, color = 'grey', linestyle = 'dashed', zorder = -1)
    ax[1].set_xlabel("")
    ax[1].set_xticklabels("")
    ax[1].legend_.remove()
    ax[1].set_ylim([0, yval])
    ax[1].yaxis.set_major_locator(ticker.MultipleLocator(ticks))
    ax[1].yaxis.set_major_formatter(ticker.ScalarFormatter())
    ax[2].set_ylabel("Plaque Percent Volume")
    ax[2].set_xlabel("")
    ax[2].legend_.remove()
    ax[2].set_ylabel("")
    ax[2].set_ylim([0, yval])
    ax[2].yaxis.set_major_locator(ticker.MultipleLocator(ticks))
    ax[2].yaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.tight_layout()
    ax[0].set_title('APP/PS1')
    ax[1].set_title('Tg2576')
    ax[2].set_title('hAPP-J20')

    plt.savefig(os.path.join(savepath, 
                             'plaque percent coverage {0} {1} boxplot.pdf'.format(title, age)), 
                bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

def make_broken_lineplot(age, structures, title, width, height, break_point, ax_max = 0, legend_loc = 2):
    pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
    for structure in structures:
        structure_acronym = ai_map[structure]
        pltdat = pd.concat([pltdat, get_structure_data(age, structure_acronym, 'density')], sort = True)
    pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]
    meddat = pltdat.groupby(['group', 'structure']).median().reset_index()
    iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
    iso = structure_tree.descendant_ids([iso['id']])[0]
    iso = [structure for structure in iso if structure in summary_structure_ids]
    if structures == iso:
        meddat['rank'] = meddat['structure'].map(sorterIndex)
        meddat = meddat.sort_values(by='rank')
    meddat['lower_error'] = pltdat.groupby(['group', 'structure']).quantile(.25).reset_index()['Percent Plaque Coverage']
    meddat['lower_error'] = meddat['Percent Plaque Coverage'] - meddat['lower_error']
    meddat['upper_error'] = pltdat.groupby(['group', 'structure']).quantile(.75).reset_index()['Percent Plaque Coverage']
    meddat['upper_error'] = meddat['upper_error'] - meddat['Percent Plaque Coverage']
    
    f, (ax, ax2) = plt.subplots(2, 1, sharex=True, figsize = (width, height))
    # plot the same data on both axes

    for ix, group in enumerate(mouse_lines):
            meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                                  ax = ax, c = mouse_line_colors[ix], linewidth = 3, capsize=.5, 
                                                  marker = 'o', alpha = 0.3, legend = False,
                                                  yerr = np.array(meddat[meddat['group'] == group][['lower_error', 'upper_error']].T))
            meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                                  ax = ax2, c = mouse_line_colors[ix], linewidth = 3, capsize=.5, 
                                                  marker = 'o', alpha = 0.3, legend = False,
                                                  yerr = np.array(meddat[meddat['group'] == group][['lower_error', 'upper_error']].T))
            meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                                  ax = ax, c = mouse_line_colors[ix], linewidth = 3,  
                                                  marker = 'o', legend = False)
            meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                                  ax = ax2, c = mouse_line_colors[ix], linewidth = 3, 
                                                  marker = 'o', legend = False)

    if ax_max == 0:
        y_max = np.max(meddat['upper_error'])
    else:
        y_max = ax_max
    # zoom-in / limit the view to different portions of the data
    ax.set_ylim(break_point, y_max)  # top half
    ax2.set_ylim(0, break_point)  # bottom half

    # hide the spines between ax and ax2
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(labeltop='off')  # don't put tick labels at the top
    ax.tick_params(top='off')
    ax2.spines['top'].set_visible(False)
    ax2.tick_params(top='off')
    ax2.xaxis.tick_bottom()

    d = .015  # how big to make the diagonal lines in axes coordinates
    # arguments to pass to plot, just so we don't keep repeating them
    kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
    ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
    ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

    kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
    ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal
    f.subplots_adjust(hspace=0.1)
    ax2.set_xlabel("")
    ax.set_title(title)
    ax.legend(handles = get_mouse_line_legend(), loc = legend_loc, frameon = False)
    plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
    plt.setp( ax2.xaxis.get_majorticklabels(), rotation=90 )
    plt.savefig(os.path.join(savepath, 'plaque percent coverage in {0} {1} lineplot broken y axis.pdf'.format(title, age)), 
                bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)
    
def make_lineplot(age, structures, title, width, height, ax_max = 0, legend_loc = 2):
    pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
    for structure in structures:
        structure_acronym = ai_map[structure]
        pltdat = pd.concat([pltdat, get_structure_data(age, structure_acronym, 'density')], sort = True)
    pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]
    meddat = pltdat.groupby(['group', 'structure']).median().reset_index()
    iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
    iso = structure_tree.descendant_ids([iso['id']])[0]
    iso = [structure for structure in iso if structure in summary_structure_ids]
    if structures == iso:
        meddat['rank'] = meddat['structure'].map(sorterIndex)
        meddat = meddat.sort_values(by='rank')
    meddat['lower_error'] = pltdat.groupby(['group', 'structure']).quantile(.25).reset_index()['Percent Plaque Coverage']
    meddat['lower_error'] = meddat['Percent Plaque Coverage'] - meddat['lower_error']
    meddat['upper_error'] = pltdat.groupby(['group', 'structure']).quantile(.75).reset_index()['Percent Plaque Coverage']
    meddat['upper_error'] = meddat['upper_error'] - meddat['Percent Plaque Coverage']
    fig, ax = plt.subplots(figsize = (width, height))
    for ix, group in enumerate(mouse_lines):
        meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                              ax = ax, c = mouse_line_colors[ix], linewidth = 3,  
                                              marker = 'o', legend = False)
        meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = mouse_line_colors[ix], linewidth = 3, capsize=.5,
                                          yerr = np.array(meddat[meddat['group'] == group][['lower_error', 'upper_error']].T), 
                                          legend = False, alpha = 0.3)
    ax.set_ylabel('Plaque Density Per Structure')
    if ax_max == 0:
        y_max = np.max(meddat['upper_error'])
    else:
        y_max = ax_max
    ax.set_ylim(0, y_max)
    plt.title(title)
    plt.legend(handles = get_mouse_line_legend(), loc = legend_loc, frameon = False)
    plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
    ax.set_xlabel("")
    plt.savefig(os.path.join(savepath, 'plaque percent coverage in {0} {1} lineplot.pdf'.format(title, age)), 
                bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

## Isocortex plotted three ways

In [ ]:
iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
iso = structure_tree.descendant_ids([iso['id']])[0]
iso = [structure for structure in iso if structure in summary_structure_ids]

In [ ]:
plot_order = ['FRP', 'MOp', 'MOs', 'SSp-n', 'SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-ul', 'SSp-tr', 'SSp-un',
              'SSs', 'GU', 'VISC', 'AUDd', 'AUDp', 'AUDpo', 'AUDv', 'VISal', 'VISam', 'VISl', 'VISp', 'VISpl',
              'VISpm', 'VISli', 'VISpor', 'ACAd', 'ACAv', 'PL', 'ILA', 'ORBl', 'ORBm', 'ORBvl', 'AId', 'AIp',
              'AIv', 'RSPagl', 'RSPd', 'RSPv', 'VISa', 'VISrl', 'TEa', 'PERI', 'ECT']

In [ ]:
sorterIndex = dict(zip(plot_order,range(len(plot_order))))

In [ ]:
make_boxplots('13 mo', iso, 'Isocortex', 8, 4, 0.35)

In [ ]:
make_boxplots('19 mo', iso, 'Isocortex', 8, 4, 1.5)

pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('13 mo', structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]
meddat = pltdat.groupby(['group', 'structure']).median().reset_index()
meddat['lower_error'] = pltdat.groupby(['group', 'structure']).quantile(.25).reset_index()['Percent Plaque Coverage']
meddat['upper_error'] = pltdat.groupby(['group', 'structure']).quantile(.75).reset_index()['Percent Plaque Coverage']
meddat['rank'] = meddat['structure'].map(sorterIndex)
meddat = meddat.sort_values(by='rank')
fig, ax = plt.subplots(figsize = (8, 4))
meddat[meddat['group'] == 'APP/PS1'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = '#8da0cb', linewidth = 3, capsize=.5,
                                          yerr = [meddat[meddat['group'] == 'APP/PS1']['lower_error'].values, 
                                                  meddat[meddat['group'] == 'APP/PS1']['upper_error'].values], 
                                          legend = False, alpha = 0.3)
meddat[meddat['group'] == 'APP/PS1'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = '#8da0cb', linewidth = 3, 
                                          legend = False)
ax2 = plt.twinx()
for group in ['hAPP-J20', 'Tg2576']:
    if group != 'APP/PS1':
        if group == 'hAPP-J20':
            color = '#fc8d62'
        if group == 'Tg2576':
            color = '#66c2a5'
        meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                              ax = ax2, c = color, linewidth = 3, capsize=.5, marker = 'o',
                                              yerr = [meddat[meddat['group'] == group]['lower_error'].values, 
                                                      meddat[meddat['group'] == group]['upper_error'].values], 
                                              legend = False, alpha = 0.3)
        meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                              ax = ax2, c = color, linewidth = 3, marker = 'o',
                                            legend = False)
ax.set_ylabel('APP/PS1')
ax2.set_ylabel('hAPP-J20, Tg2576')
plt.title('Plaque Density in Isocortex Structures')
plt.legend(handles = get_mouse_line_legend(), loc = 2, frameon = False)
plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
ax.set_xlabel("")
#plt.savefig(os.path.join(savepath, 'plaque percent coverage in isocortex structures 12 mo lineplot two axes.pdf'.format(structure)), 
#            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
make_broken_lineplot('13 mo', iso, 'Isocortex', 9, 4, 0.23, 2)

In [ ]:
make_lineplot('19 mo', iso, 'Isocortex', 9, 4, 1.8)

## Hippocampus plotted three ways

In [ ]:
hipp = st.get_structures_by_acronym(['HPF'])[0]
hipp_ids = st.descendant_ids([hipp['id']])[0]
hipp_ss = [ids for ids in hipp_ids if ids in summary_structure_ids]
hipp_ss.remove(ia_map['IG'])
print(len(hipp_ss))

In [ ]:
make_boxplots('13 mo', hipp_ss, 'Hippocampus', 3.5, 4, 0.4)

In [ ]:
make_boxplots('19 mo', hipp_ss, 'Hippocampus', 3.5, 4, 1.3)

In [ ]:
make_broken_lineplot('13 mo', hipp_ss, 'Hippocampus', 3, 4, 0.23, 2,)

In [ ]:
make_lineplot('13 mo', hipp_ss, 'Hippocampus', 3, 4, 1.4)

In [ ]:
make_lineplot('19 mo', hipp_ss, 'Hippocampus', 3, 4, 1.6, 1)

## Olfactory regions plotted three ways

In [ ]:
olf = st.get_structures_by_acronym(['OLF'])[0]
olf_ids = st.descendant_ids([olf['id']])[0]
olf_ss = [ids for ids in olf_ids if ids in summary_structure_ids]
print(len(olf_ss))

In [ ]:
make_boxplots('13 mo', olf_ss, 'Olfactory Structures', 3, 4, 0.35)

In [ ]:
make_boxplots('19 mo', olf_ss, 'Olfactory Structures', 3, 4, 0.6)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in olf_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('13 mo', structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]
meddat = pltdat.groupby(['group', 'structure']).median().reset_index()
mindat = pltdat.groupby(['group', 'structure']).quantile(.25).reset_index()
maxdat = pltdat.groupby(['group', 'structure']).quantile(.75).reset_index()
meddat['error'] = maxdat['Percent Plaque Coverage'] - mindat['Percent Plaque Coverage']

fig, ax = plt.subplots(figsize = (3, 4))
meddat[meddat['group'] == 'APP/PS1'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = '#8da0cb', linewidth = 3, capsize=.5,
                                          yerr = 'error', alpha = 0.3, legend = False)
meddat[meddat['group'] == 'APP/PS1'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = '#8da0cb', linewidth = 3, legend = False)
ax2 = plt.twinx()
for group in ['hAPP-J20', 'Tg2576']:
    if group != 'APP/PS1':
        if group == 'hAPP-J20':
            color = '#fc8d62'
        if group == 'Tg2576':
            color = '#66c2a5'
        meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                              ax = ax2, c = color, linewidth = 3, capsize=.5, marker = 'o',
                                              yerr = 'error', alpha = 0.3, legend = False)
        meddat[meddat['group'] == group].plot('structure', 'Percent Plaque Coverage', 
                                              ax = ax2, c = color, linewidth = 3, marker = 'o',
                                              legend = False)
ax.set_ylabel('APP/PS1')
ax2.set_ylabel('hAPP-J20, Tg2576')
ax.set_title('Plaque Density in Olfactory Structures')
plt.legend(handles = get_mouse_line_legend(), loc = 2, frameon = False)
plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
plt.savefig(os.path.join(savepath, 'plaque percent coverage in olfactory structures 13 mo lineplot two axes.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
make_broken_lineplot('13 mo', olf_ss, 'Olfactory Structures', 2.5, 4, 0.09, 1.7)

In [ ]:
make_lineplot('19 mo', olf_ss, 'Olfactory Structures', 2.5, 4, 2, 1)

## CTXsp structures plotted three ways

In [ ]:
ctxsp = st.get_structures_by_acronym(['CTXsp'])[0]
ctxsp_ids = st.descendant_ids([ctxsp['id']])[0]
ctxsp_ss = [ids for ids in ctxsp_ids if ids in summary_structure_ids]
print(len(ctxsp_ss))

In [ ]:
make_boxplots('13 mo', ctxsp_ss, 'Cortical Subplate', 2, 4, 0.2)

In [ ]:
make_boxplots('19 mo', ctxsp_ss, 'Cortical Subplate', 2, 4, 0.6)

In [ ]:
make_broken_lineplot('13 mo', ctxsp_ss, 'Cortical Subplate', 2, 4, 0.12, 2.2)

In [ ]:
make_broken_lineplot('19 mo', ctxsp_ss, 'Cortical Subplate', 2, 4, 0.2, 2.5)

In [ ]:
make_lineplot('19 mo', ctxsp_ss, 'Cortical Subplate', 2, 4, 2.2)

## Plot by age

In [ ]:
def get_age_legend(colors, ages):
    legend = {}
    for ix, age in enumerate(ages):
        legend[ix] = mlines.Line2D([], [], color=colors[ix],
                              label=age)
    return [value for key, value in legend.iteritems()]

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
for age in ages:
    for structure in iso:
        structure_acronym = ai_map[structure]
        pltdat = pd.concat([pltdat, get_structure_data(age, structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]

fig, ax = plt.subplots(figsize = (8, 3))
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
colors = ['#d0d1e6', '#a6bddb', '#74a9cf', '#2b8cbe', '#045a8d']
for ix, age in enumerate(ages):
    agedat = pltdat[pltdat['age'] == age]
    meddat = agedat.groupby(['group', 'structure']).median().reset_index()
    mapping = {region: i for i, region in enumerate(plot_order)}
    key = meddat['structure'].map(mapping)    
    meddat = meddat.iloc[key.argsort()]
    meddat['lower_error'] = agedat.groupby(['group', 'structure']).quantile(.25).reset_index()['Percent Plaque Coverage']
    meddat['lower_error'] = meddat['Percent Plaque Coverage'] - meddat['lower_error']
    meddat['upper_error'] = agedat.groupby(['group', 'structure']).quantile(.75).reset_index()['Percent Plaque Coverage']
    meddat['upper_error'] = meddat['upper_error'] - meddat['Percent Plaque Coverage']
    meddat[meddat['group'] == 'APP/PS1'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, capsize=.5,
                                              alpha = 0.3, legend = False,
                                          yerr = np.array(meddat[meddat['group'] == 'APP/PS1'][['lower_error', 'upper_error']].T)) 
    meddat[meddat['group'] == 'APP/PS1'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, legend = False)
plt.title('Plaque Density in Isocortex Structures APP/PS1')
plt.legend(handles = get_age_legend(colors, ages), loc = 2)
plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
ax.set_xlabel("")
ax.set_ylabel("Percent Plaque Coverage")
ax.set_ylim([-0.05, 1.9])
plt.savefig(os.path.join(savepath, 'plaque volume in isocortex by age APP_PS1.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
for age in ages:
    for structure in hipp_ss:
        structure_acronym = ai_map[structure]
        pltdat = pd.concat([pltdat, get_structure_data(age, structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]

fig, ax = plt.subplots(figsize = (3, 3))
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
colors = ['#d0d1e6', '#a6bddb', '#74a9cf', '#2b8cbe', '#045a8d']
for ix, age in enumerate(ages):
    agedat = pltdat[pltdat['age'] == age]
    meddat = agedat.groupby(['group', 'structure']).median().reset_index()
    meddat['lower_error'] = agedat.groupby(['group', 'structure']).quantile(.25).reset_index()['Percent Plaque Coverage']
    meddat['lower_error'] = meddat['Percent Plaque Coverage'] - meddat['lower_error']
    meddat['upper_error'] = agedat.groupby(['group', 'structure']).quantile(.75).reset_index()['Percent Plaque Coverage']
    meddat['upper_error'] = meddat['upper_error'] - meddat['Percent Plaque Coverage']
    meddat[meddat['group'] == 'APP/PS1'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, capsize=.5,
                                              alpha = 0.3, legend = False,
                                          yerr = np.array(meddat[meddat['group'] == 'APP/PS1'][['lower_error', 'upper_error']].T)) 
    meddat[meddat['group'] == 'APP/PS1'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, legend = False)
plt.title('Plaque Density in Hippocampal Structures APP/PS1')
plt.legend(handles = get_age_legend(colors, ages), loc = 1)
plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
ax.set_xlabel("")
ax.set_ylabel("Percent Plaque Coverage")
ax.set_ylim([-0.05, 1.9])
plt.savefig(os.path.join(savepath, 'plaque volume in hippocampus by age APP_PS1.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
for age in ages:
    for structure in iso:
        structure_acronym = ai_map[structure]
        pltdat = pd.concat([pltdat, get_structure_data(age, structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]

fig, ax = plt.subplots(figsize = (8, 3))
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
colors = ['#fdd49e', '#fdbb84', '#fc8d59', '#e34a33', '#b30000']
for ix, age in enumerate(ages):
    agedat = pltdat[pltdat['age'] == age]
    meddat = agedat.groupby(['group', 'structure']).median().reset_index()
    mapping = {region: i for i, region in enumerate(plot_order)}
    key = meddat['structure'].map(mapping)    
    meddat = meddat.iloc[key.argsort()]
    meddat['lower_error'] = agedat.groupby(['group', 'structure']).quantile(.25).reset_index()['Percent Plaque Coverage']
    meddat['lower_error'] = meddat['Percent Plaque Coverage'] - meddat['lower_error']
    meddat['upper_error'] = agedat.groupby(['group', 'structure']).quantile(.75).reset_index()['Percent Plaque Coverage']
    meddat['upper_error'] = meddat['upper_error'] - meddat['Percent Plaque Coverage']
    meddat[meddat['group'] == 'hAPP-J20'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, capsize=.5,
                                              alpha = 0.3, legend = False,
                                          yerr = np.array(meddat[meddat['group'] == 'hAPP-J20'][['lower_error', 'upper_error']].T)) 
    meddat[meddat['group'] == 'hAPP-J20'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, legend = False)
plt.title('Plaque Density in Isocortex Structures hAPP-J20')
plt.legend(handles = get_age_legend(colors, ages), loc = 2)
plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
ax.set_xlabel("")
ax.set_ylabel("Percent Plaque Coverage")
ax.set_ylim([-0.05, 0.7])
plt.savefig(os.path.join(savepath, 'plaque volume in isocortex by age J20.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
for age in ages:
    for structure in hipp_ss:
        structure_acronym = ai_map[structure]
        pltdat = pd.concat([pltdat, get_structure_data(age, structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]

fig, ax = plt.subplots(figsize = (3, 3))
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
colors = ['#fdd49e', '#fdbb84', '#fc8d59', '#e34a33', '#b30000']
for ix, age in enumerate(ages):
    agedat = pltdat[pltdat['age'] == age]
    meddat = agedat.groupby(['group', 'structure']).median().reset_index()
    meddat['lower_error'] = agedat.groupby(['group', 'structure']).quantile(.25).reset_index()['Percent Plaque Coverage']
    meddat['lower_error'] = meddat['Percent Plaque Coverage'] - meddat['lower_error']
    meddat['upper_error'] = agedat.groupby(['group', 'structure']).quantile(.75).reset_index()['Percent Plaque Coverage']
    meddat['upper_error'] = meddat['upper_error'] - meddat['Percent Plaque Coverage']
    meddat[meddat['group'] == 'hAPP-J20'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, capsize=.5,
                                              alpha = 0.3, legend = False,
                                          yerr = np.array(meddat[meddat['group'] == 'hAPP-J20'][['lower_error', 'upper_error']].T)) 
    meddat[meddat['group'] == 'hAPP-J20'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, legend = False)
plt.title('Plaque Density in Hippocampal Structures hAPP-J20')
plt.legend(handles = get_age_legend(colors, ages), loc = 1)
plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
ax.set_xlabel("")
ax.set_ylabel("Percent Plaque Coverage")
ax.set_ylim([-0.05, 0.7])
plt.savefig(os.path.join(savepath, 'plaque volume in hippocampus by age J20.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
ages = ['5 mo', '7 mo', '9 mo', '13 mo', '19 mo']
for age in ages:
    for structure in olf_ss:
        structure_acronym = ai_map[structure]
        pltdat = pd.concat([pltdat, get_structure_data(age, structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]

fig, ax = plt.subplots(figsize = (8, 4))
ages = ['9 mo', '13 mo', '19 mo']
colors = ['#e5f5f9', '#99d8c9', '#2ca25f']
for ix, age in enumerate(ages):
    agedat = pltdat[pltdat['age'] == age]
    meddat = agedat.groupby(['group', 'structure']).median().reset_index()
    meddat['lower_error'] = agedat.groupby(['group', 'structure']).quantile(.25).reset_index()['Percent Plaque Coverage']
    meddat['lower_error'] = meddat['Percent Plaque Coverage'] - meddat['lower_error']
    meddat['upper_error'] = agedat.groupby(['group', 'structure']).quantile(.75).reset_index()['Percent Plaque Coverage']
    meddat['upper_error'] = meddat['upper_error'] - meddat['Percent Plaque Coverage']
    meddat[meddat['group'] == 'Tg2576'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, capsize=.5,
                                              alpha = 0.3, legend = False,
                                          yerr = np.array(meddat[meddat['group'] == 'Tg2576'][['lower_error', 'upper_error']].T)) 
    meddat[meddat['group'] == 'Tg2576'].plot('structure', 'Percent Plaque Coverage', 
                                          ax = ax, c = colors[ix], linewidth = 3, legend = False)
plt.title('Plaque Density in Olfactory Structures Tg2576')
plt.legend(handles = get_age_legend(colors, ages), loc = 2, frameon = False)
plt.xticks(np.arange(len(meddat['structure'].unique())), meddat['structure'].unique());
plt.setp( ax.xaxis.get_majorticklabels(), rotation=90 )
ax.set_xlabel("")
plt.savefig(os.path.join(savepath, 'plaque volume in olfactory by age Tg2576.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

## Plaque density by module

In [ ]:
dat.loc[dat['structure_acronym'].isin(['FRP', 'MOs', 'PL', 'ILA', 'ORBm', 'ORBl', 'ORBvl','ACAd', 'ACAv']),
        'module'] = 'Prefrontal'
dat.loc[dat['structure_acronym'].isin(['AId', 'AIv', 'AIp', 'GU', 'VISC']), 'module'] = 'Anterolateral'
dat.loc[dat['structure_acronym'].isin(['SSs', 'SSp-bfd', 'SSp-tr', 'SSp-ll', 'SSp-ul', 'SSp-un',
                                          'SSp-n', 'SSp-m', 'MOp']), 'module'] = 'Somatomotor'
dat.loc[dat['structure_acronym'].isin(['VISp', 'VISl', 'VISrl', 'VISpl', 'VISli', 'VISal', 'VISpor']), 
        'module'] = 'Visual'
dat.loc[dat['structure_acronym'].isin(['RSPd', 'RSPv', 'RSPagl', 'VISa', 'VISam', 'VISpm']), 'module'] = 'Medial'
dat.loc[dat['structure_acronym'].isin(['AUDd', 'AUDp', 'AUDpo', 'AUDv', 'TEa', 'PERI', 'ECT']), 'module'] = 'Temporal'

In [ ]:
len(dat[dat['module'] == 'Temporal']['structure_acronym'].unique())

In [ ]:
dat.head()

In [ ]:
dat['module'].unique()

In [ ]:
moduledat = pd.DataFrame()
for mouse_line in dat[dat['mouse_line'].isin(['APP/PS1', 'Tg2576', 'hAPP-J20'])]['mouse_line'].unique():
    modules = []
    mods = []
    ctxdat = dat[(dat['mouse_line'] == mouse_line) & 
                 (~dat['module'].isnull())]
    sumdat = ctxdat['plaque_density'].sum()
    for module in ctxdat['module'].unique():
        modules.append(module)
        mods.append(float(np.sum(ctxdat[(ctxdat['mouse_line'] == mouse_line) & 
                      (ctxdat['module'] == module)]['plaque_density']))/sumdat)
    moduledat[mouse_line] = mods
moduledat['Module'] = modules

In [ ]:
moduledat

In [ ]:
mdf = pd.melt(moduledat[['Module', 'APP/PS1', 'hAPP-J20', 'Tg2576']], 
              id_vars = ['Module'], var_name = ['Mouse Line'], value_name = 'Fraction of Cortical Plaque Density')

In [ ]:
mdf.head()

In [ ]:
g = sns.catplot('Module', 'Fraction of Cortical Plaque Density', data = mdf, hue = 'Mouse Line', kind='bar',
            palette = mouse_line_colors, hue_order = mouse_lines,
           height = 5, aspect = 1.1, legend = False)
g.ax.legend(loc = 2)
g.ax.axhline(y=0.15, color = 'grey', linestyle = 'dashed', zorder = -1)
plt.savefig(os.path.join(savepath, 'plaque density by module.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
def get_mean_value_per_structure(group, age, structure_ids):
    means = []
    isids = dataset[(dataset['mouse_line'] == group) & (dataset['age_group'] == age)]['image_series_id'].values
    for structure_id in structure_ids:
        str_mean = np.mean(dat[(dat['structure_id'] == structure_id) & 
                                 (dat['image_series_id'].isin(isids))]
                             ['plaque_density'])
        means.append(str_mean*100)
    structuredat = dict(zip(structure_ids, means))
    return structuredat, len(isids)

In [ ]:
all_structure_vals, n = get_mean_value_per_structure('Tg2576', '13 mo', dat.structure_id.unique())

In [ ]:
reference_space =  mcc.get_reference_space()
structure_vals, n = get_mean_value_per_structure('APP/PS1', '13 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 0.7
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
#index = [8000, 4500, 5000]
index = [6650, 5625, 9100] #rostral
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'APP_PS1 plaque map 13 mo rostral.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('APP/PS1', '19 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 0.7
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
#index = [8000, 4500, 5000]
index = [6650, 5625, 9100] #rostral
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'APP_PS1 plaque map 19 mo rostral.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('hAPP-J20', '19 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 2.5
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
#index = [8000, 4500, 5000]
index = [6650, 5625, 9100] #rostral
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'hAPP-J20 plaque map 19 mo rostral.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
structure_vals, n = get_mean_value_per_structure('hAPP-J20', '13 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 5
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
#index = [8000, 4500, 5000]
index = [6650, 5625, 9100] #rostral
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), 
                         np.round(maxval*.34, 2), np.round(maxval*.45, 2), 
                         np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'hAPP-J20 plaque map 13 mo rostral.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('Tg2576', '13 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 10
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
#index = [8000, 4500, 5000]
index = [6650, 5625, 9100] #rostral
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), 
                         np.round(maxval*.34, 2), np.round(maxval*.45, 2), 
                         np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'Tg2576 plaque map 13 mo rostral.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
structure_vals, n = get_mean_value_per_structure('Tg2576', '19 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 2
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
#index = [8000, 4500, 5000]
index = [6650, 5625, 9100] #rostral
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor= rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')

cbar = fig.colorbar(f, orientation='vertical', 
                    fraction=0.046)
cbar_ax = fig.axes[-1]
cbar.solids.set_rasterized(True)
cbar.solids.set_edgecolor("face")
cbar.draw_all()

cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'Tg2576 plaque map 19 mo rostral.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
w=8
h=8
image = [0,0,0]
#index = [8000, 4500, 5000]
index = [6650, 5625, 9100] #rostral
image[0] = reference_space.get_slice_image(0, index[0])
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2])), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1]))  
fig = plt.figure(figsize=(8, 3), facecolor= rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i])
    plt.axis('off')
cbar = fig.colorbar(f, orientation='vertical', 
                    fraction=0.046)
cbar_ax = fig.axes[-1]
cbar.solids.set_rasterized(True)
cbar.solids.set_edgecolor("face")
cbar.draw_all()

cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
plt.savefig(os.path.join(savepath, 'ARA with plaque map rostral.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
meandat = dat.groupby('structure_acronym').mean().reset_index().sort_values(by = 'plaque_density', ascending = False)

In [ ]:
cdat = pd.read_csv(r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque/figures/control brains/control_false_positive_mean_by_ss.csv')

In [ ]:
meandat.sort_values(by='structure_id', inplace = True)
meandat['Percent Coverage'] = [density*100 for density in meandat['plaque_density']]

In [ ]:
cdat.sort_values(by='structure_id', inplace = True)
cdat['Percent Coverage'] = [density*100 for density in cdat['plaque_density']]

In [ ]:
diff = meandat['Percent Coverage'] - cdat['Percent Coverage']

In [ ]:
max(diff)

In [ ]:
min(diff)

In [ ]:
np.mean(diff)

In [ ]:
fig, ax = plt.subplots(figsize = (2, 2.2))
g = sns.distplot(diff, kde=0)
g.set_ylabel('Number of Structures')
g.set_xlabel('Diff in % Coverage')
g.set_title('Signal - False Positive')
plt.tight_layout()
plt.savefig(r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque/figures/control brains/false_positive_levels.pdf', 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
len(diff)

In [ ]:
len([value for value in diff if value < 0.1])

In [ ]:
len([value for value in diff if -0.1 < value < 0.1])

In [ ]:
len([value for value in diff if value < 0])

In [ ]:
len([value for value in diff if value > 0.5])

In [ ]:
max(diff)

In [ ]:
dat.head()

In [ ]:
iso_children = st.child_ids(iso)
iso_children = [item for sublist in iso_children for item in sublist]

In [ ]:
len(iso_children)

In [ ]:
hipp['id']

In [ ]:
hipp_children = st.descendant_ids([hipp['id']])
hipp_children = [item for sublist in hipp_children for item in sublist]

In [ ]:
dat.head()

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure_acronym in dat['structure_acronym'].unique():
    pltdat = pd.concat([pltdat, get_structure_data('19 mo', structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]

In [ ]:
maxstr = pltdat.groupby('image_series_id').max()

In [ ]:
medplaque = pltdat[pltdat['group'] == 'APP/PS1']['Percent Plaque Coverage'].quantile(0.95)

In [ ]:
medplaque

In [ ]:
max_strs = []
for isid in pltdat[pltdat['group'] == 'APP/PS1']['image_series_id'].unique():
    isid_dat = pltdat[pltdat['image_series_id'] == isid]
    max_strs.append(isid_dat[isid_dat['Percent Plaque Coverage'] > medplaque]['structure'].values)

In [ ]:
max_strs = [item for sublist in max_strs for item in sublist]

In [ ]:
set(max_strs)

In [ ]:
pltdat[(pltdat['group'] == 'APP/PS1') & 
              (pltdat['structure'] == 'cc')]['Percent Plaque Coverage'].mean()

In [ ]:
pltdat[(pltdat['group'] == 'APP/PS1')].describe()

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure_acronym in ['Isocortex']:
    pltdat = pd.concat([pltdat, get_structure_data('9 mo', structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]

In [ ]:
pltdat[(pltdat['group'] == 'Tg2576')].describe()

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in [hipp['id']]:
    print(structure)
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('19 mo', structure_acronym, 'density')], sort = True)
pltdat['Percent Plaque Coverage'] = [density*100 for density in pltdat['plaques']]

In [ ]:
iso_children